# Exploratory Data Analysis
We will move our exploratory analysis in this separate notebook, 
in order to remove a lot of code clutter from our main notebook.

After we have cleaned our data and they are in a format that we are confortable with, we will save them, 
and access them from our main notebook, where we will continue working on them.

In [2]:
# imports
import pandas as pd

# import the datetime library to convert out movie_budget['release_date'] column to datetime datatype
from datetime import datetime

From our data, we will be using four datasets:
* imdb.title.basic located at `/data/title.basics.csv`
* imdb.title.crew located at `/data/title.crew.csv`
* imdb.title.ratings located at `/data/title.ratings.csv`
* imdb.tn.movie_budgets located at `/data/tn.movie_budgets.csv`

In [3]:
# loading our data
title_basics = pd.read_csv('./data/title.basics.csv')
title_crew = pd.read_csv('./data/title.crew.csv')
title_ratings = pd.read_csv('./data/title.ratings.csv')
movie_budgets = pd.read_csv('./data/tn.movie_budgets.csv')

Take a look at the data we have loaded to get an idea of the kind of data we will be working with,

and clean them where needed, so that we have a neatly cleaned and formatted data for working with.

## Cleaning IMDB Title Basics
We now focus on the `title_basics` dataset, to clean the data, 
and get all the columns in the desired formats and types

In [4]:
#title basic top 5
title_basics.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [5]:
# Check the general information about our dataset
title_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


* Our dataset has 146144 entries,
* `original_title`, `runtime_minutes`, `genres` have some missing values,
* `start_year` is in the form of integers as expected.

In [6]:
# take a look at the year column to see the values contained within 
title_basics.start_year.value_counts()

2017    17504
2016    17272
2018    16849
2015    16243
2014    15589
2013    14709
2012    13787
2011    12900
2010    11849
2019     8379
2020      937
2021       83
2022       32
2023        5
2024        2
2027        1
2026        1
2025        1
2115        1
Name: start_year, dtype: int64

We will drop the `original_title` since we will not be using it and the `primary_title`, 
has the same content as it, and convert all values in `primary_title` to lowercase.

We will also drop the entries with years beyond the current year 2022.

In [7]:
# create a copy of title basics that we can change without affecting the base table
# and drop the original_title column
title_basics_cleaned = title_basics.drop('original_title', axis=1).copy()

# drop any row that has an empty/NaN cell
title_basics_cleaned = title_basics_cleaned.dropna(axis=0, how='any')

# drop all rows in the year column that have year greater than 2022
title_basics_cleaned.drop(title_basics_cleaned[title_basics_cleaned['start_year'] > 2022].index, inplace = True)

# convert all the entries in our primary column to lower case
title_basics_cleaned['primary_title'] = title_basics_cleaned['primary_title'].map(lambda x: x.lower())

In [8]:
# check the general information about our dataset, to see if our changes took effect.
title_basics_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112233 entries, 0 to 146139
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           112233 non-null  object 
 1   primary_title    112233 non-null  object 
 2   start_year       112233 non-null  int64  
 3   runtime_minutes  112233 non-null  float64
 4   genres           112233 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 10.1+ MB


In [9]:
# save the cleaned file for further exploration in the main notebook
title_basics_cleaned.to_csv('./data/cleaned_title_basics.csv', index=False)

## Cleaning IMDB Title Crew

We now look at the crew data, to see its formats, and check if there is anything we need to clean

In [10]:
# check the top 5 entries of title_crew
title_crew.head()

,tconst,directors,writers
0,tt0285252,nm0899854,nm0899854
1,tt0438973,NaN,"nm0175726,nm1802864"
2,tt0462036,nm1940585,nm1940585
3,tt0835418,nm0151540,"nm0310087,nm0841532"
4,tt0878654,"nm0089502,nm2291498,nm2292011",nm0284943


In [11]:
# check the general information about the dataframe
title_crew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   tconst     146144 non-null  object
 1   directors  140417 non-null  object
 2   writers    110261 non-null  object
dtypes: object(3)
memory usage: 3.3+ MB


We won't make any deletions to our `title_crew` data, as the table ties both directors and writers together,
and deleting a row because of missing a director, will lead to us deleting the writers as well, and vice versa.

Instead we'll fill the empty cells with `Unknown`.

In [12]:
# fill all the empty cell of the dataframe with the term 'unknown'
title_crew = title_crew.fillna('unknown')

In [13]:
# check if the changes have been effected
title_crew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   tconst     146144 non-null  object
 1   directors  146144 non-null  object
 2   writers    146144 non-null  object
dtypes: object(3)
memory usage: 3.3+ MB


In [14]:
# save our modified dataset for further exploration later
title_crew.to_csv('./data/cleaned_title_crew.csv', index=False)

## Cleaning Title Ratings

We now look at the title ratings to see if our dataset is in the desired format, 
and make a decision on the empty cells

In [15]:
# check the first 5 values of our dataframe
title_ratings.head()

,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [16]:
# check our general dataset information
title_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


The title ratings has no empty cells, and all the columns are in a desired format.

However, ratings with few numvotes(number of votes) tend to be biased, which in turn
greatly affects our ratings, and so, we will only work with rating with atleast 10 votes.

After which, we'll save the data in a new dataset, for consistency.

In [17]:
# drop all rows in the ratings table, that have num votes less than 20
title_ratings.drop(title_ratings[title_ratings['numvotes'] < 10].index, inplace = True)

In [18]:
# check the general information about our dataset, to see if our changes took effect.
title_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61710 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         61710 non-null  object 
 1   averagerating  61710 non-null  float64
 2   numvotes       61710 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.9+ MB


In [19]:
# save our dataset for further exploration later
title_ratings.to_csv('./data/cleaned_title_ratings.csv', index=False)

## Cleaning The Number's Movie budgets

We now consider the Numbers movie budget, and check if there is any cleaning needed, 
and any data formating needed.

In [20]:
# Check the toop 5 values of our dataset
movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [21]:
# check the general info about our dataset.
movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


We dont have any missing values in our dataset, however, the dataset is mostly in the string format, 
and thus not useful to work with yet.

We'll make a few changes to the datset, including:
* Get rid of the `id` column, as it's a repetition and not visually neat.
* Convert the `release_date` to a date datatype.
* Convert the `movie` to all lower case, to enable matching all movie, case not withstanding
* Convert the `production_budget`, `domestic_gross`, `worldwide_gross` to integer, and get rid of the dollar sign and commas.

### Additional feature
We'll add a few columns to our movie_budgets dataset, including
* `approx_profit` - difference between the production_budget and worldwide_gross
* `approx_profit_perc` - approx_profit as a percentage of the production_budget
* `year_of_release` - the year of release
* `month_of_release` - the month of release

In [22]:
# drop the id column of movie_ratings
movie_budgets.drop('id', axis=1, inplace=True)

In the code cell below, we will use the datetime library to help us with converting our column to a datatype

In [23]:
# Convert movie_budgets['release_date'] column into a date dtype
movie_budgets['release_date'] = movie_budgets['release_date'].map(lambda x: datetime.strptime(x, '%b %d, %Y'))


In [24]:
# check the top 5 items to see if changes on our release date column took effect.
movie_budgets.head()

,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,2009-12-18,Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2011-05-20,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,2019-06-07,Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,2015-05-01,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,2017-12-15,Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


we will then work on converting our movie column to all lowercase, 
to counter for string that might have been written in mixedcase or uppercase

In [25]:
# using .map() convert the column to lower case.
movie_budgets['movie'] = movie_budgets['movie'].map(lambda x: x.lower())

In [26]:
# check to see our changes took effect on the movie column
movie_budgets.head()

,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,2009-12-18,avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2011-05-20,pirates of the caribbean: on stranger tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,2019-06-07,dark phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,2015-05-01,avengers: age of ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,2017-12-15,star wars ep. viii: the last jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [27]:
# we now work on removing the dolar sign, commars, and converting the data type to integer
# we will write a functio to do the above

def clean_column(df, column_to_clean, item_to_clean):
    """A function that takes in a dataset, column, item
        and removes the item from the specified column
    """
    return df[column_to_clean].map(lambda x: x.replace(f'{item_to_clean}', ""))

In [28]:
# remove the dollar signs
movie_budgets['production_budget'] = clean_column(movie_budgets, 'production_budget', '$')
movie_budgets['domestic_gross'] = clean_column(movie_budgets, 'domestic_gross', '$')
movie_budgets['worldwide_gross'] = clean_column(movie_budgets, 'worldwide_gross', '$')

# remove the comma
movie_budgets['production_budget'] = clean_column(movie_budgets, 'production_budget', ',')
movie_budgets['domestic_gross'] = clean_column(movie_budgets, 'domestic_gross', ',')
movie_budgets['worldwide_gross'] = clean_column(movie_budgets, 'worldwide_gross', ',')

# convert the datatype
movie_budgets['production_budget'] = movie_budgets['production_budget'].astype('int64')
movie_budgets['domestic_gross'] = movie_budgets['domestic_gross'].astype('int64')
movie_budgets['worldwide_gross'] = movie_budgets['worldwide_gross'].astype('int64')

In [29]:
# check to see if our changes took effect
movie_budgets.head()

,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,2009-12-18,avatar,425000000,760507625,2776345279
1,2011-05-20,pirates of the caribbean: on stranger tides,410600000,241063875,1045663875
2,2019-06-07,dark phoenix,350000000,42762350,149762350
3,2015-05-01,avengers: age of ultron,330600000,459005868,1403013963
4,2017-12-15,star wars ep. viii: the last jedi,317000000,620181382,1316721747


In [30]:
# check to see if our data is now in the desired format
movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   release_date       5782 non-null   datetime64[ns]
 1   movie              5782 non-null   object        
 2   production_budget  5782 non-null   int64         
 3   domestic_gross     5782 non-null   int64         
 4   worldwide_gross    5782 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 226.0+ KB


Now that we have converted our columns to integers, we can check to see the ones with value 0, and delete them from our dataset.

We will altogether delete any item with either a domestic or worldwide gross below 1000

In [31]:
# drop rows with any of the columns less than 1000
movie_budgets.drop(movie_budgets[movie_budgets['production_budget'] < 1000].index, inplace = True)
movie_budgets.drop(movie_budgets[movie_budgets['domestic_gross'] < 1000].index, inplace = True)
movie_budgets.drop(movie_budgets[movie_budgets['worldwide_gross'] < 1000].index, inplace = True)

### Feature Addition
At this point, we now need to add our new columns to the dataset

In [32]:
# include all 0f our additional features into our dataset

# find the approximate profit by subtracting the production budget from the worldwide gross.
movie_budgets['approx_profit'] = movie_budgets['worldwide_gross'] - movie_budgets['production_budget']

# find the approximate profit percentage, by calculating approximate profit as a percentage of the production budget
movie_budgets['approx_profit_perc'] = round((movie_budgets['approx_profit'] / movie_budgets['production_budget'])*100, 2)

# find the year of release
movie_budgets['year_of_release'] = pd.DatetimeIndex(movie_budgets['release_date']).year

# find the month of release
movie_budgets['month_of_release'] = pd.DatetimeIndex(movie_budgets['release_date']).month

In [33]:
# check to see that our changes took effect
movie_budgets.head()

,release_date,movie,production_budget,domestic_gross,worldwide_gross,approx_profit,approx_profit_perc,year_of_release,month_of_release
0,2009-12-18,avatar,425000000,760507625,2776345279,2351345279,553.26,2009,12
1,2011-05-20,pirates of the caribbean: on stranger tides,410600000,241063875,1045663875,635063875,154.67,2011,5
2,2019-06-07,dark phoenix,350000000,42762350,149762350,-200237650,-57.21,2019,6
3,2015-05-01,avengers: age of ultron,330600000,459005868,1403013963,1072413963,324.38,2015,5
4,2017-12-15,star wars ep. viii: the last jedi,317000000,620181382,1316721747,999721747,315.37,2017,12


In [34]:
# Check the general information of our new dataset.
movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5222 entries, 0 to 5781
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   release_date        5222 non-null   datetime64[ns]
 1   movie               5222 non-null   object        
 2   production_budget   5222 non-null   int64         
 3   domestic_gross      5222 non-null   int64         
 4   worldwide_gross     5222 non-null   int64         
 5   approx_profit       5222 non-null   int64         
 6   approx_profit_perc  5222 non-null   float64       
 7   year_of_release     5222 non-null   int64         
 8   month_of_release    5222 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(6), object(1)
memory usage: 568.0+ KB


In [35]:
# save our dataset for further exploration later
movie_budgets.to_csv('./data/cleaned_movie_budgets.csv', index=False)

We have now cleaned four datasets that we will be using in our primary notebook